In [1]:
!ls /workspace/results/syn_baseline_v2/

dataflow_partition0_6ozjxmlt		    folding_config.json
dataflow_partition0_w509dzku		    folding_config_fifos.json
dataflow_partition1_2uh492jg		    librispeech_data
dataflow_partition2_i_cp5_2l		    outputs_test.npy
end2end_quartznet_dataflow_partition.onnx   partitioning_lowering
end2end_quartznet_export_dev.onnx	    partitioning_repartition
end2end_quartznet_folded.onnx		    qce_alveo_v2
end2end_quartznet_hls_layers.onnx	    qce_alveo_v2.zip
end2end_quartznet_lowered.onnx		    vitis_link_proj_ohf619xn
end2end_quartznet_lowered_partitioned.onnx  vitis_link_proj_qce_alveo_v2
end2end_quartznet_streamline.onnx	    vitis_link_proj_qce_alveo_v2.zip
end2end_quartznet_tidy.onnx


In [2]:
!ls /workspace/results/debug_rtlsim/

df_model.onnx		       parent_model.onnx
end2end_quartznet_rtlsim.onnx  partition_0.onnx


# Pre-process inputs

In [25]:
import numpy as np

a = np.load("/workspace/results/librispeech_data/test_input_1.npy")

print(np.shape(a))

a1 = a[:,:,0:256]
a2 = a[:,:,256:512]
a3 = a[:,:,512:768]
a4 = a[:,:,768:]
a4 = np.pad(a4, ((0,0),(0,0),(0,128)))

print(np.shape(a1))
print(np.shape(a2))
print(np.shape(a3))
print(np.shape(a4))



(1, 64, 896)
(1, 64, 256)
(1, 64, 256)
(1, 64, 256)
(1, 64, 256)


In [26]:
import finn.core.onnx_exec as oxe
from finn.core.modelwrapper import ModelWrapper
from finn.core.datatype import DataType

model = ModelWrapper("/workspace/results/syn_baseline_v3/end2end_quartznet_bitfile.onnx")
iname = model.graph.input[0].name

inp_val = a4

inp_val = np.reshape(inp_val, np.shape(inp_val)+(1,)) # Make 4D tensor
input_dict = {iname: inp_val}
# Change tensor annotation to FLOAT for simulation (must be kept INT otherwise InferDataTypes will annotate all tensors as FLOAT)
first_conv_out = model.graph.node[0].output[0]
for t in [iname, first_conv_out]:
    model.set_tensor_datatype(t, DataType.FLOAT32)
# Find index of StreamingDataflowPartition node
sdf_node = model.get_nodes_by_op_type("StreamingDataflowPartition")
assert(len(sdf_node)==1), "Found more than 1 StreamingDataflowPartition node!"
sdf_node_idx = model.get_node_index(sdf_node[0])
output_dict = oxe.execute_onnx(model, input_dict, return_full_exec_context=True, end_node=model.graph.node[sdf_node_idx-1])
# Execute IPstitched part of graph
sdf_input = sdf_node[0].input[0]
inp_val_sdf = output_dict[sdf_input]

np.save("/workspace/results/librispeech_data/input_sample_1_pt3.npy", inp_val_sdf)

# Visual inspection

In [6]:
from finn.util.visualization import showInNetron

showInNetron("/workspace/results/syn_optimized_buffer/end2end_quartznet_rtlsim.onnx")

Serving '/workspace/results/syn_optimized_buffer/end2end_quartznet_rtlsim.onnx' at http://0.0.0.0:8081


In [9]:
from finn.util.visualization import showInNetron
showInNetron("/workspace/results/syn_baseline_v3/end2end_quartznet_bitfile.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/syn_baseline_v3/end2end_quartznet_bitfile.onnx' at http://0.0.0.0:8081


In [2]:
from finn.util.visualization import showInNetron
showInNetron("/workspace/results/debug_rtlsim/df_model_stitchedip.onnx")

Serving '/workspace/results/debug_rtlsim/df_model_stitchedip.onnx' at http://0.0.0.0:8081


In [48]:
from finn.util.visualization import showInNetron
showInNetron("/workspace/results/debug_rtlsim/wrong_datatype_model/df_model.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/debug_rtlsim/wrong_datatype_model/df_model.onnx' at http://0.0.0.0:8081


In [46]:
from finn.util.visualization import showInNetron
showInNetron("/workspace/results/debug_rtlsim/wrong_datatype_model/parent_model.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/debug_rtlsim/wrong_datatype_model/parent_model.onnx' at http://0.0.0.0:8081


In [16]:
from finn.core.modelwrapper import ModelWrapper
from finn.custom_op.registry import getCustomOp

model = ModelWrapper("/workspace/results/debug_rtlsim/wrong_datatype_model/df_model_rtlsim.onnx")

for n in model.graph.node:
    getCustomOp(n).set_nodeattr("code_gen_dir_ipgen", "")
    getCustomOp(n).set_nodeattr("ipgen_path", "")
    getCustomOp(n).set_nodeattr("ip_path", "")
    getCustomOp(n).set_nodeattr("ip_vlnv", "")
    getCustomOp(n).set_nodeattr("rtlsim_so", "")
    if n.name=="StreamingFCLayer_Batch_0":
        getCustomOp(n).set_nodeattr("SIMD",1)
    
model.save("/workspace/results/debug_rtlsim/wrong_datatype_model/parent_model.onnx")

In [23]:
showInNetron("/workspace/results/debug_rtlsim/fifos_inserted/parent_model.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/debug_rtlsim/fifos_inserted/parent_model.onnx' at http://0.0.0.0:8081


In [1]:
from finn.util.visualization import showInNetron
showInNetron("/workspace/results/debug_rtlsim/df_model_fifos.onnx")

Serving '/workspace/results/debug_rtlsim/df_model_fifos.onnx' at http://0.0.0.0:8081


In [2]:
from finn.util.visualization import showInNetron
showInNetron("/workspace/results/syn_baseline_v2/end2end_quartznet_fifos.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/syn_baseline_v2/end2end_quartznet_fifos.onnx' at http://0.0.0.0:8081


In [83]:
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.create_generic_partitions import PartitionFromDict

model = ModelWrapper("/workspace/results/debug_rtlsim/end2end_quartznet_streamline.onnx")
model = model.transform(PartitionFromDict({0:range(2,4)}))

model.save("/tmp/model_test.onnx")



In [84]:
showInNetron("/tmp/model_test.onnx")

Stopping http://0.0.0.0:8081
Serving '/tmp/model_test.onnx' at http://0.0.0.0:8081


In [85]:
showInNetron("/tmp/finn_dev_mirza/partitioning_nzoogq68/partition_0.onnx")


Stopping http://0.0.0.0:8081
Serving '/tmp/finn_dev_mirza/partitioning_nzoogq68/partition_0.onnx' at http://0.0.0.0:8081


In [143]:
from finn.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from finn.transformation.fpgadataflow.convert_to_hls_layers import InferThresholdingLayer, InferQuantizedStreamingFCLayer
from finn.transformation.infer_data_layouts import InferDataLayouts
from finn.transformation.infer_datatypes import InferDataTypes
from finn.transformation.streamline.absorb import AbsorbConsecutiveTransposes
from finn.transformation.fpgadataflow.create_dataflow_partition import CreateDataflowPartition
from finn.transformation.streamline.reorder import MoveTransposePastMultiThreshold

model = ModelWrapper("/tmp/finn_dev_mirza/partitioning_nzoogq68/partition_0.onnx")
model = model.transform(LowerConvsToMatMul())
model = model.transform(InferDataLayouts())

model = model.transform(MoveTransposePastMultiThreshold())

model = model.transform(InferDataTypes())

#model = model.transform(InferThresholdingLayer())
model = model.transform(InferQuantizedStreamingFCLayer())
#model = model.transform(AbsorbConsecutiveTransposes())

model = model.transform(CreateDataflowPartition())


model.save("/tmp/m.onnx")



0-------


In [144]:
showInNetron("/tmp/m.onnx")

Stopping http://0.0.0.0:8081
Serving '/tmp/m.onnx' at http://0.0.0.0:8081


In [145]:
from finn.custom_op.registry import getCustomOp
from finn.transformation.general import GiveUniqueNodeNames, GiveReadableTensorNames

showInNetron("/tmp/finn_dev_mirza/dataflow_partition0_615h3_gx/df_model.onnx")

model = ModelWrapper("/tmp/finn_dev_mirza/dataflow_partition0_615h3_gx/df_model.onnx")

model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())

for n in model.graph.node:
    if n.op_type=="StreamingFCLayer_Batch":
        getCustomOp(n).set_nodeattr("PE", 8)
        getCustomOp(n).set_nodeattr("SIMD", 1)
        getCustomOp(n).set_nodeattr("resType", 'dsp')
        getCustomOp(n).set_nodeattr("mem_mode", 'decoupled')
        getCustomOp(n).set_nodeattr("ram_style", 'distributed')
    elif n.op_type=="Thresholding_Batch":
        getCustomOp(n).set_nodeattr("PE", 1)
        getCustomOp(n).set_nodeattr("mem_mode", 'const')
        getCustomOp(n).set_nodeattr("ram_style", 'distributed')

model.save("/workspace/results/debug_rtlsim/single_layer/mvau/streamingfc.onnx")

Stopping http://0.0.0.0:8081
Serving '/tmp/finn_dev_mirza/dataflow_partition0_615h3_gx/df_model.onnx' at http://0.0.0.0:8081


In [147]:
showInNetron("/workspace/results/debug_rtlsim/single_layer/mvau/streamingfc.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/debug_rtlsim/single_layer/mvau/streamingfc.onnx' at http://0.0.0.0:8081


In [150]:
showInNetron("/workspace/results/debug_rtlsim/single_layer/mvau_threshold/df_model_rtlsim.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/debug_rtlsim/single_layer/mvau_threshold/df_model_rtlsim.onnx' at http://0.0.0.0:8081


# Prepare to partition model

In [82]:
from finn.util.visualization import showInNetron
showInNetron("/workspace/results/syn_baseline_v2/dataflow_partition0_6ozjxmlt/df_model.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/syn_baseline_v2/dataflow_partition0_6ozjxmlt/df_model.onnx' at http://0.0.0.0:8081


In [83]:
model = ModelWrapper("/workspace/results/syn_baseline_v2/dataflow_partition0_6ozjxmlt/df_model.onnx")
for idx,n in enumerate(model.graph.node):
    print("{}\t{}".format(n.name, idx))

StreamingFCLayer_Batch_0	0
DuplicateStreams_Batch_0	1
FMPadding_Batch_0	2
StreamingFCLayer_Batch_1	3
ConvolutionInputGenerator1D_0	4
Vector_Vector_Activate_Batch_0	5
StreamingFCLayer_Batch_2	6
FMPadding_Batch_1	7
ConvolutionInputGenerator1D_1	8
Vector_Vector_Activate_Batch_1	9
StreamingFCLayer_Batch_3	10
FMPadding_Batch_2	11
ConvolutionInputGenerator1D_2	12
Vector_Vector_Activate_Batch_2	13
StreamingFCLayer_Batch_4	14
FMPadding_Batch_3	15
ConvolutionInputGenerator1D_3	16
Vector_Vector_Activate_Batch_3	17
StreamingFCLayer_Batch_5	18
FMPadding_Batch_4	19
ConvolutionInputGenerator1D_4	20
Vector_Vector_Activate_Batch_4	21
StreamingFCLayer_Batch_6	22
AddStreams_Batch_0	23
Thresholding_Batch_0	24
DuplicateStreams_Batch_1	25
FMPadding_Batch_5	26
StreamingFCLayer_Batch_7	27
ConvolutionInputGenerator1D_5	28
Vector_Vector_Activate_Batch_5	29
StreamingFCLayer_Batch_8	30
FMPadding_Batch_6	31
ConvolutionInputGenerator1D_6	32
Vector_Vector_Activate_Batch_6	33
StreamingFCLayer_Batch_9	34
FMPadding_Ba

In [87]:
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.create_generic_partitions import PartitionFromDict

model = ModelWrapper("/workspace/results/syn_baseline_v2/dataflow_partition0_6ozjxmlt/df_model.onnx")
model = model.transform(PartitionFromDict({0:range(0,25)}))

model.save("/tmp/model_test.onnx")



In [89]:
showInNetron("/tmp/finn_dev_mirza/partitioning_fjjwwt54/partition_0.onnx")

Stopping http://0.0.0.0:8081
Serving '/tmp/finn_dev_mirza/partitioning_fjjwwt54/partition_0.onnx' at http://0.0.0.0:8081


In [92]:
from finn.custom_op.registry import getCustomOp
model = ModelWrapper("/tmp/finn_dev_mirza/partitioning_fjjwwt54/partition_0.onnx")

for n in model.graph.node:
    getCustomOp(n).set_nodeattr("code_gen_dir_ipgen", "")
    getCustomOp(n).set_nodeattr("ipgen_path", "")
    getCustomOp(n).set_nodeattr("ip_path", "")
    getCustomOp(n).set_nodeattr("ip_vlnv", "")
    if n.name=="StreamingFCLayer_Batch_0":
        getCustomOp(n).set_nodeattr("SIMD", 2)

model.save("/workspace/results/debug_rtlsim/df_model.onnx")

In [93]:
showInNetron("/workspace/results/debug_rtlsim/df_model.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/debug_rtlsim/df_model.onnx' at http://0.0.0.0:8081


In [16]:
model = ModelWrapper("/workspace/results/syn_baseline_v2/end2end_quartznet_fifos.onnx")

sdp_node = model.get_nodes_by_op_type("StreamingDataflowPartition")[0]
getCustomOp(sdp_node).set_nodeattr("model", "/workspace/results/df_model.onnx")

model.save("/workspace/results/debug_rtlsim/parent_model.onnx")

In [17]:
showInNetron("/workspace/results/debug_rtlsim/parent_model.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/debug_rtlsim/parent_model.onnx' at http://0.0.0.0:8081


# RUN first

In [1]:
def binary_to_int(bitstream, pack=8, signed=True):
    int_stream=[]
    for i in range(pack):
        bit = bitstream[8*i:8*(i+1)]
        if signed:
            int_val = -2**7 * int(bit[0]) + 2**6 * int(bit[1]) + 2**5 * int(bit[2]) + 2**4 * int(bit[3]) + 2**3 * int(bit[4]) + 2**2 * int(bit[5]) + 2**1 * int(bit[6]) + 2**0 * int(bit[7])
        else:
            int_val = 2**7 * int(bit[0]) + 2**6 * int(bit[1]) + 2**5 * int(bit[2]) + 2**4 * int(bit[3]) + 2**3 * int(bit[4]) + 2**2 * int(bit[5]) + 2**1 * int(bit[6]) + 2**0 * int(bit[7])
        int_stream.append(int_val)
    return(int_stream)

def hex_to_binary(hex_string):
    bits = ''
    for el in hex_string:
        if el=='0':
            bits+= '0000'
        elif el=='1':
            bits += '0001'
        elif el=='2':
            bits += '0010'
        elif el=='3':
            bits += '0011'
        elif el=='4':
            bits += '0100'
        elif el=='5':
            bits += '0101'
        elif el=='6':
            bits += '0110'
        elif el=='7':
            bits += '0111'
        elif el=='8':
            bits += '1000'
        elif el=='9':
            bits += '1001'
        elif el=='A':
            bits += '1010'
        elif el=='B':
            bits += '1011'
        elif el=='C':
            bits += '1100'
        elif el=='D':
            bits += '1101'
        elif el=='E':
            bits += '1110'
        elif el=='F':
            bits += '1111'
    return bits
            

In [17]:
showInNetron("/workspace/results/debug_rtlsim/single_layer/df_model_rtlsim.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/debug_rtlsim/single_layer/df_model_rtlsim.onnx' at http://0.0.0.0:8081


# Debug weights

In [10]:
import numpy as np
from finn.core.modelwrapper import ModelWrapper

model = ModelWrapper("/workspace/results/debug_rtlsim/wrong_datatype_model/df_model_rtlsim.onnx")

first_node = model.graph.node[0]

weight_first_node = model.get_initializer(first_node.input[1])

[1,128,1,64] -> [-5, 3, -4]

print(np.shape(weight_first_node))

print(weight_first_node[0,0:8])
print(weight_first_node[1,0:8])
print(weight_first_node[2,0:8])
print(weight_first_node[3,0:8])
print(weight_first_node[4,0:8])
print(weight_first_node[5,0:8])
print(weight_first_node[6,0:8])
print(weight_first_node[7,0:8])
print(weight_first_node[8,0:8])
print(weight_first_node[9,0:8])
print(weight_first_node[10,0:8])

....

print(weight_first_node[63,0:8])

print(weight_first_node[0,8:15])


(64, 256)
[  3.  -9. -25.  -2.   3.   3.  11.  -4.]
[ 2.  7. 38. -9. -5. -3.  3.  5.]
[  3.  -5. -61.   1. -11. -14. -11.  17.]
[  6. -14.  -8.  -3.   4.  -2.  -5. -15.]
[ -1.   2. -41. -19. -28.  -8.   2.  23.]
[   0.    7.   11.  -49. -127.   -7.  -27.  -39.]
[ -8.  -3.  -2.  -3. -39.   2.   7.  22.]
[ -1.   1.  69. 127. -21.   8.  -6. -13.]
[  0. -15. -90.  23.  16.  -7.  27.   7.]
[  -2.   33. -118.   42.   -4.  -20.  -56.  -14.]
[ 11.  -1. 127. -29.  -2.  18. -53.  -2.]


In [15]:
w0 = binary_to_int(hex_to_binary('FC0B0303FEE7F703'), 8, True)[::-1]
print(w0)

w0 = binary_to_int(hex_to_binary('0503FDFBF7260702'), 8, True)[::-1]
print(w0)

w0 = binary_to_int(hex_to_binary('11F5F2F501C3FB03'), 8, True)[::-1]
print(w0)

w0 = binary_to_int(hex_to_binary('F1FBFE04FDF8F206'), 8, True)[::-1]
print(w0)

w0 = binary_to_int(hex_to_binary('1702F8E4EDD702FF'), 8, True)[::-1]
print(w0)

w0 = binary_to_int(hex_to_binary('D9E5F981CF0B0700'), 8, True)[::-1]
print(w0)

w0 = binary_to_int(hex_to_binary('160702D9FDFEFDF8'), 8, True)[::-1]
print(w0)

w0 = binary_to_int(hex_to_binary('F3FA08EB7F4501FF'), 8, True)[::-1]
print(w0)

w0 = binary_to_int(hex_to_binary('F2C8ECFC2A8A21FE'), 8, True)[::-1]
print(w0)

w0 = binary_to_int(hex_to_binary('FECB12FEE37FFF0B'), 8, True)[::-1]
print(w0)

[3, -9, -25, -2, 3, 3, 11, -4]
[2, 7, 38, -9, -5, -3, 3, 5]
[3, -5, -61, 1, -11, -14, -11, 17]
[6, -14, -8, -3, 4, -2, -5, -15]
[-1, 2, -41, -19, -28, -8, 2, 23]
[0, 7, 11, -49, -127, -7, -27, -39]
[-8, -3, -2, -3, -39, 2, 7, 22]
[-1, 1, 69, 127, -21, 8, -6, -13]
[-2, 33, -118, 42, -4, -20, -56, -14]
[11, -1, 127, -29, -2, 18, -53, -2]


# Debug thresholds

In [32]:
import numpy as np
from finn.core.modelwrapper import ModelWrapper

model = ModelWrapper("/workspace/results/debug_rtlsim/wrong_datatype_model/df_model_rtlsim.onnx")

first_node = model.graph.node[0]

threshold_first_node = model.get_initializer(first_node.input[2])

print(np.shape(threshold_first_node))

print(threshold_first_node[0,0:8])
print(threshold_first_node[1,0:4])



(256, 255)
[-11623. -11289. -10955. -10621. -10287.  -9953.  -9619.  -9285.]
[-16618. -16012. -15407. -14801.]


In [34]:
t0 = int(hex_to_binary('08272'),2)
print(t0)

np.argwhere(threshold_first_node==t0)

33394


array([[ 7, 95]])

# Check global/node output

In [13]:
import numpy as np
cpp_out = np.load("/workspace/results/debug_rtlsim/cppsim_out_same_input_simd2.npy", allow_pickle=True).item()
#print(cpp_out.keys())
cpp_out_global = cpp_out["Thresholding_Batch_0_out0"]
print(np.shape(cpp_out_global))
print(cpp_out_global[0,0,0,:])
#print(cpp_out_global[0,1,0,:])


(1, 128, 1, 256)
[10.  2.  3.  3.  0.  0.  2.  0.  5.  2.  2.  0.  0.  2.  3.  2.  2.  0.
  0.  0.  3.  3.  5.  0.  0.  5.  7.  0.  0.  0.  0. 10.  2.  5.  0.  0.
  0.  0.  7.  0.  5.  2.  2.  0.  0.  0.  0.  0.  0.  2.  2. 12.  3.  2.
  5.  0.  0.  3.  2. 15. 10.  2.  5.  2.  2.  3.  0.  5.  2.  3.  0.  0.
  0.  0.  8.  0.  2.  2.  2.  2.  7.  0.  0.  0.  8.  5.  0.  0.  5.  0.
 10.  0.  2.  2.  2.  0.  3.  0.  0.  7.  3.  3.  5.  0.  0.  0.  5.  2.
  2.  0.  0.  3.  2.  0. 15.  0.  3.  0. 10.  0.  0.  3.  3.  2.  0.  2.
  0.  3.  3.  0.  0.  7.  2.  0.  0.  0.  2.  0.  0.  0.  2.  0.  0. 10.
  3.  5.  0.  0.  3.  0.  0.  8.  0.  2.  0.  2.  5. 10.  5.  0.  0.  3.
  0.  0.  5.  5.  5.  0.  3.  0.  5.  2.  0.  3.  2.  0.  0.  0.  0.  0.
  2.  2. 10.  3.  2.  2.  0.  0.  0.  0.  2.  0.  0.  5.  0.  0.  5.  0.
  3.  0.  2.  0.  0.  0.  0.  0.  3.  0.  0.  5.  3.  0.  0.  0.  3.  0.
  2.  3.  0.  3.  0.  2.  5.  0.  0.  2.  0.  0.  0.  3.  0.  0.  0.  0.
  2.  0.  0.  0.  0.  0.  0.  5.  

In [9]:
### OLD CPP sim model
import numpy as np
cpp_out1 = np.load("/workspace/results/debug_rtlsim/quartznet_cppsim_old.npy", allow_pickle=True).item()
#print(cpp_out1.keys())
cpp_out_global1 = cpp_out1["_QEN4Re"]
print(np.shape(cpp_out_global1))
print(cpp_out_global1[0,0,0,:])
#print(cpp_out_global[0,1,0,:])


(1, 128, 1, 256)
[10.  2.  3.  3.  0.  0.  2.  0.  5.  2.  2.  0.  0.  2.  3.  2.  2.  0.
  0.  0.  3.  3.  5.  0.  0.  5.  7.  0.  0.  0.  0. 10.  2.  5.  0.  0.
  0.  0.  7.  0.  5.  2.  2.  0.  0.  0.  0.  0.  0.  2.  2. 12.  3.  2.
  5.  0.  0.  3.  2. 15. 10.  2.  5.  2.  2.  3.  0.  5.  2.  3.  0.  0.
  0.  0.  8.  0.  2.  2.  2.  2.  7.  0.  0.  0.  8.  5.  0.  0.  5.  0.
 10.  0.  2.  2.  2.  0.  3.  0.  0.  7.  3.  3.  5.  0.  0.  0.  5.  2.
  2.  0.  0.  3.  2.  0. 15.  0.  3.  0. 10.  0.  0.  3.  3.  2.  0.  2.
  0.  3.  3.  0.  0.  7.  2.  0.  0.  0.  2.  0.  0.  0.  2.  0.  0. 10.
  3.  5.  0.  0.  3.  0.  0.  8.  0.  2.  0.  2.  5. 10.  5.  0.  0.  3.
  0.  0.  5.  5.  5.  0.  3.  0.  5.  2.  0.  3.  2.  0.  0.  0.  0.  0.
  2.  2. 10.  3.  2.  2.  0.  0.  0.  0.  2.  0.  0.  5.  0.  0.  5.  0.
  3.  0.  2.  0.  0.  0.  0.  0.  3.  0.  0.  5.  3.  0.  0.  0.  3.  0.
  2.  3.  0.  3.  0.  2.  5.  0.  0.  2.  0.  0.  0.  3.  0.  0.  0.  0.
  2.  0.  0.  0.  0.  0.  0.  5.  

In [10]:
import numpy as np
cpp_out = np.load("/workspace/results/debug_rtlsim/cppsim_out_same_input_simd2.npy", allow_pickle=True).item()
#print(cpp_out.keys())
cpp_out_node = cpp_out["StreamingFCLayer_Batch_0_out0"]
cpp_out_global = cpp_out["global_out"]
print(np.shape(cpp_out_node))
print(np.shape(cpp_out_global))
print(cpp_out_global[0,0,0,:])


(1, 128, 1, 256)
(1, 128, 1, 256)
[10.  2.  3.  3.  0.  0.  2.  0.  5.  2.  2.  0.  0.  2.  3.  2.  2.  0.
  0.  0.  3.  3.  5.  0.  0.  5.  7.  0.  0.  0.  0. 10.  2.  5.  0.  0.
  0.  0.  7.  0.  5.  2.  2.  0.  0.  0.  0.  0.  0.  2.  2. 12.  3.  2.
  5.  0.  0.  3.  2. 15. 10.  2.  5.  2.  2.  3.  0.  5.  2.  3.  0.  0.
  0.  0.  8.  0.  2.  2.  2.  2.  7.  0.  0.  0.  8.  5.  0.  0.  5.  0.
 10.  0.  2.  2.  2.  0.  3.  0.  0.  7.  3.  3.  5.  0.  0.  0.  5.  2.
  2.  0.  0.  3.  2.  0. 15.  0.  3.  0. 10.  0.  0.  3.  3.  2.  0.  2.
  0.  3.  3.  0.  0.  7.  2.  0.  0.  0.  2.  0.  0.  0.  2.  0.  0. 10.
  3.  5.  0.  0.  3.  0.  0.  8.  0.  2.  0.  2.  5. 10.  5.  0.  0.  3.
  0.  0.  5.  5.  5.  0.  3.  0.  5.  2.  0.  3.  2.  0.  0.  0.  0.  0.
  2.  2. 10.  3.  2.  2.  0.  0.  0.  0.  2.  0.  0.  5.  0.  0.  5.  0.
  3.  0.  2.  0.  0.  0.  0.  0.  3.  0.  0.  5.  3.  0.  0.  0.  3.  0.
  2.  3.  0.  3.  0.  2.  5.  0.  0.  2.  0.  0.  0.  3.  0.  0.  0.  0.
  2.  0.  0.  0. 

In [16]:
import numpy as np
#rtlsim_out = np.load("/workspace/results/debug_rtlsim/rtlsim_out_same_input.npy", allow_pickle=True).item()
#rtlsim_out = np.load("/workspace/results/debug_rtlsim/single_layer/mvau_threshold/rtlsim_out.npy", allow_pickle=True).item()
rtlsim_out = np.load("/workspace/results/debug_rtlsim/fifos_inserted/quartznet_sdp_node.npy", allow_pickle=True).item()
#rtlsim_out = np.load("/workspace/results/debug_rtlsim/wrong_datatype_model/quartznet_sdp_node.npy", allow_pickle=True).item()

node_out_rtl = rtlsim_out["global_out"]

print(node_out_rtl[0,0,0,:])

print((cpp_out_global==node_out_rtl).all())

[10.  2.  3.  3.  0.  0.  2.  0.  5.  2.  2.  0.  0.  2.  3.  2.  2.  0.
  0.  0.  3.  3.  5.  0.  0.  5.  7.  0.  0.  0.  0. 10.  2.  5.  0.  0.
  0.  0.  7.  0.  5.  2.  2.  0.  0.  0.  0.  0.  0.  2.  2. 12.  3.  2.
  5.  0.  0.  3.  2. 15. 10.  2.  5.  2.  2.  3.  0.  5.  2.  3.  0.  0.
  0.  0.  8.  0.  2.  2.  2.  2.  7.  0.  0.  0.  8.  5.  0.  0.  5.  0.
 10.  0.  2.  2.  2.  0.  3.  0.  0.  7.  3.  3.  5.  0.  0.  0.  5.  2.
  2.  0.  0.  3.  2.  0. 15.  0.  3.  0. 10.  0.  0.  3.  3.  2.  0.  2.
  0.  3.  3.  0.  0.  7.  2.  0.  0.  0.  2.  0.  0.  0.  2.  0.  0. 10.
  3.  5.  0.  0.  3.  0.  0.  8.  0.  2.  0.  2.  5. 10.  5.  0.  0.  3.
  0.  0.  5.  5.  5.  0.  3.  0.  5.  2.  0.  3.  2.  0.  0.  0.  0.  0.
  2.  2. 10.  3.  2.  2.  0.  0.  0.  0.  2.  0.  0.  5.  0.  0.  5.  0.
  3.  0.  2.  0.  0.  0.  0.  0.  3.  0.  0.  5.  3.  0.  0.  0.  3.  0.
  2.  3.  0.  3.  0.  2.  5.  0.  0.  2.  0.  0.  0.  3.  0.  0.  0.  0.
  2.  0.  0.  0.  0.  0.  0.  5.  3.  0.  0.  0.  7

In [38]:
rtlsim_out = np.load("/workspace/results/debug_rtlsim/rtlsim_out_same_input_simd2.npy", allow_pickle=True).item()

#rtlsim_out = np.load("/workspace/results/debug_rtlsim/rtlsim_out_same_input.npy", allow_pickle=True).item()

node_out_rtl = rtlsim_out["global_out"]

print(node_out_rtl[0,0,0,:])



[10.  0.  0.  3.  0.  0.  5.  0.  8.  0.  3.  0.  0.  2.  5.  2.  3.  0.
  0.  0.  3.  3.  7.  0.  0.  5.  7.  2.  0.  0.  5. 10.  0.  0.  2.  0.
  0.  0.  7.  0.  5.  2.  0.  0.  3.  0.  0.  0.  0.  3.  3. 10.  2.  2.
  5.  0.  0.  3.  3. 15. 10.  3.  5.  2.  0.  3.  0.  3.  2.  3.  0.  0.
  2.  0.  3.  0.  2.  2.  0.  2.  7.  0.  0.  0.  8.  3.  0.  0.  3.  0.
 10.  0.  3.  0.  2.  0.  3.  0.  0.  7.  3.  5.  5.  0.  0.  0.  3.  0.
  0.  0.  0.  2.  3.  0. 12.  0.  3.  0. 10.  0.  0.  3.  3.  0.  0.  0.
  2.  3.  0.  0.  0.  8.  0.  0.  2.  0.  2.  0.  0.  0.  2.  0.  2.  8.
  3.  5.  0.  2.  3.  0.  0.  8.  3.  0.  0.  3.  3. 10.  5.  0.  0.  3.
  0.  0.  5.  5.  5.  0.  0.  0.  5.  5.  0.  3.  0.  0.  0.  0.  0.  0.
  3.  0. 10.  2.  0.  0.  0.  2.  0.  0.  2.  0.  0.  5.  0.  0.  3.  0.
  2.  0.  2.  0.  0.  0.  2.  2.  3.  0.  0.  5.  0.  0.  0.  0.  3.  0.
  2.  5.  0.  3.  0.  2.  5.  0.  0.  0.  0.  0.  0.  3.  0.  0.  2.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  7

In [126]:
print((node_out==node_out_rtl).all())


False


# Debug inputs

In [2]:
print(binary_to_int(hex_to_binary('03FB'),2)[::-1])
print(binary_to_int(hex_to_binary('FAFC'),2)[::-1])
print(binary_to_int(hex_to_binary('FCFC'),2)[::-1])
print(binary_to_int(hex_to_binary('FCFD'),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])
#print(binary_to_int(hex_to_binary(''),2)[::-1])

[-5, 3]
[-4, -6]
[-4, -4]
[-3, -4]


In [4]:
import numpy as np
in_tensor = np.load("/workspace/results/ntnu_debug/cppsim_sdp_in.npy")
in_tensor_cppsim = np.load("/workspace/results/ntnu_debug/cppsim_out.npy", allow_pickle=True).item()['global_in']
in_tensor_rtlsim = np.load("/workspace/results/ntnu_debug/rtlsim_out.npy", allow_pickle=True).item()['global_in']

print(np.shape(in_tensor_rtlsim))
print(in_tensor_rtlsim[0,0,0,:])

-5, -5, 3, -4, -6, -4, -4, -3, -4, 2, 2, -3, 2, -2, 2, -2, 0, -1, 2,
0, 3, 0, 0, 1, 2, 0, 1, 0, -1, -1, 2, 2, 2, 4, -2, 3, -3,
2, 1, -4, -2, 3, 3, -1, -4, 1, 2, -2, -3, 2, 2, -2, -3, -2, 1, 
2, -2, -1, -2, 0, -2, 0, -1, -4, 0, -21



(1, 128, 1, 64)
[-5.  3. -4. -6. -4. -4. -3. -4.  2.  2. -3.  2. -2.  2. -2.  0. -1.  2.
  0.  3.  0.  0.  1.  2.  0.  1.  0. -1. -1.  2.  2.  2.  4. -2.  3. -3.
  2.  1. -4. -2.  3.  3. -1. -4.  1.  2. -2. -3.  2.  2. -2. -3. -2.  1.
  2. -2. -1. -2.  0. -2.  0. -1. -4.  0.]


# Debug node outputs

In [18]:
showInNetron("/workspace/results/debug_rtlsim/single_layer/mvau/df_model_rtlsim.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/debug_rtlsim/single_layer/mvau/df_model_rtlsim.onnx' at http://0.0.0.0:8081


In [40]:
cpp_out = np.load("/workspace/results/debug_rtlsim/cppsim_out_same_input_simd2.npy", allow_pickle=True).item()
#print(cpp_out.keys())
node_out = cpp_out["StreamingFCLayer_Batch_0_out0"]
print(np.shape(node_out))
print(node_out[0,0,0,:])
#print(node_out[0,1,0,:])


(1, 128, 1, 256)
[ 35.  28.   0.   0.   0.   7.  43.  17.   4.   3.   9.  24.  17.   9.
  31.  26.   1.   0.   5.  13.   1.  18.   1.   0.  35.   0.  22.   0.
   4.  19.   3.   4.   0.  11.   0.   0.  18.  43.   6.  28.  10.  31.
   0.  26.  20.  26.   0.   0.   2.   2.   0.   8.   0.  25.  15.  16.
   0.   3.  27.  29.  22.   0.  40.  30.   0.  34.  27.  13.   1.   0.
   6.   3.   0.  21.  21.   0.  30.  14.  40.  23.  20.   7.  17.   0.
  15.   0.   0.  21.   0.   1.  24.   7.  31.  15.   0.  87.   6.  46.
   0.   0.  21.  39.   1.   0.   0.   1.   5.  18.  41.  39.   2.   0.
   4.   4.   1.   0.  23.   4.  15.  28.   5.  32.  11.  13.   5.   0.
  38.   4.   0.   0.  15.  20.   0.  20.   4.  38.   6.  31.   9.  31.
   2.   6.   0.   0.  32.   0.  11.  12.  35.   1.   0.   0.   0.   0.
   4.  14.   0.   0.  30.  29.  20.   2.  25.   1.   4.   3.   9.  25.
  35.  17.  22.  15.   0.   7.   1.   0.  20.  22.   0.   0.   3. 102.
   1.  17.   5.  14.  23.  45.   5.   0.  29.  31.   0.   0.

In [39]:
output = []

#wtest = binary_to_int(hex_to_binary('112B070000001C24'), pack=8)
#wtest = binary_to_int(hex_to_binary('102D070000001A23'), pack=8)
wtest = binary_to_int(hex_to_binary('112B070000001C23'), pack=8)
print(wtest[::-1])
output.append(wtest[::-1])

#wtest = binary_to_int(hex_to_binary('1A1E071118080103'), pack=8)
#wtest = binary_to_int(hex_to_binary('1A1D0911190A0201'), pack=8)
wtest = binary_to_int(hex_to_binary('1A1F091118090304'), pack=8)
print(wtest[::-1])
output.append(wtest[::-1])

#wtest = binary_to_int(hex_to_binary('0405120109010003'), pack=8)
#wtest = binary_to_int(hex_to_binary('0202150510060001'), pack=8)
wtest = binary_to_int(hex_to_binary('000112010D050001'), pack=8)
print(wtest[::-1])
output.append(wtest[::-1])

#wtest = binary_to_int(hex_to_binary('0400160500170024'), pack=8)
#wtest = binary_to_int(hex_to_binary('0500140700170021'), pack=8)
wtest = binary_to_int(hex_to_binary('0403130400160023'), pack=8)
print(wtest[::-1])
output.append(wtest[::-1])

#wtest = binary_to_int(hex_to_binary('19032A1400010B00'), pack=8)
#wtest = binary_to_int(hex_to_binary('1A052A1300000C00'), pack=8)
wtest = binary_to_int(hex_to_binary('1C062B1200000B00'), pack=8)
print(wtest[::-1])
output.append(wtest[::-1])

print(output)

[35, 28, 0, 0, 0, 7, 43, 17]
[4, 3, 9, 24, 17, 9, 31, 26]
[1, 0, 5, 13, 1, 18, 1, 0]
[35, 0, 22, 0, 4, 19, 3, 4]
[0, 11, 0, 0, 18, 43, 6, 28]
[[35, 28, 0, 0, 0, 7, 43, 17], [4, 3, 9, 24, 17, 9, 31, 26], [1, 0, 5, 13, 1, 18, 1, 0], [35, 0, 22, 0, 4, 19, 3, 4], [0, 11, 0, 0, 18, 43, 6, 28]]


In [67]:
import numpy as np
from finn.core.modelwrapper import ModelWrapper
from finn.custom_op.general.multithreshold import multithreshold

model = ModelWrapper("/workspace/results/debug_rtlsim/wrong_datatype_model/df_model_rtlsim.onnx")

first_node = model.graph.node[0]

in_tensor = np.load("/workspace/results/ntnu_debug/cppsim_sdp_in.npy")
weight_first_node = model.get_initializer(first_node.input[1])
threshold_first_node = model.get_initializer(first_node.input[2])

results_matmul = np.matmul(in_tensor, weight_first_node)
results_tp = np.transpose(results_matmul, [0,3,1,2])
results_mt = multithreshold(results_tp, threshold_first_node)
results_final = np.transpose(results_mt, [0,2,3,1])

In [75]:
print(results_matmul[0,0,0,0:8])

[ -93. -155. -729.  -27.  761.  127.  172.   32.]


# RTL Sim verification

In [4]:
labels = [" ", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m",
         "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "'"]
    
def __ctc_decoder_predictions_tensor(tensor, labels):
    """
    Decodes a sequence of labels to words
    """
    blank_id = len(labels)
    hypotheses = []
    labels_map = dict([(i, labels[i]) for i in range(len(labels))])
    decoded_prediction = []
    previous = len(labels)  # id of a blank symbol
    prediction = tensor
    for p in prediction:
        if (p != previous or previous == blank_id) and p != blank_id:
            decoded_prediction.append(p)
        previous = p
    hypothesis = ''.join([labels_map[c] for c in decoded_prediction])
    hypotheses.append(hypothesis)
    return hypotheses

In [66]:
#out_streamline = dict(np.load("/workspace/results/syn_baseline/librispeech_data/quartznet_streamline.npy", allow_pickle=True).tolist())
out_cpp = dict(np.load("/workspace/results/syn_baseline/librispeech_data/quartznet_cppsim.npy", allow_pickle=True).tolist())

np.save("/tmp/cppsim_sdp_out.npy",out_cpp["MUfsj8"])
np.save("/tmp/cppsim_sdp_in.npy", out_cpp["W4Ez6D"])

In [7]:
#rtlsim_out = np.load("/workspace/results/syn_baseline_v2/librispeech_data/quartznet_stitched_ip_rtlsim.npy", allow_pickle=True).item()


print(rtlsim_out)

{'global_in': array([[[[ -5.,   3.,  -4., ...,  -1.,  -4.,   0.]],

        [[-21.,  30., -10., ...,  -2.,  -4.,  -5.]],

        [[  1., -15., -29., ..., -10., -14.,  -5.]],

        ...,

        [[-25., -20.,  85., ...,  22.,  25.,  32.]],

        [[-53.,  30.,  39., ...,  31.,  14.,  35.]],

        [[-26., -20.,  72., ...,  15.,  15.,  36.]]]], dtype=float32), 'global_out': array([[[[4.9290000e+03, 1.6777168e+07, 1.6776673e+07, ...,
          1.6773812e+07, 1.6774645e+07, 3.2580000e+03]],

        [[4.1860000e+03, 1.6130000e+03, 1.6775606e+07, ...,
          1.6773958e+07, 1.6773882e+07, 3.0290000e+03]],

        [[4.1410000e+03, 1.0580000e+03, 1.6776125e+07, ...,
          1.6774172e+07, 1.6773658e+07, 3.2900000e+03]],

        ...,

        [[1.6774567e+07, 1.6774880e+07, 1.6776221e+07, ...,
          2.3560000e+03, 1.6775410e+07, 5.7380000e+03]],

        [[1.6774584e+07, 1.6774833e+07, 1.6776778e+07, ...,
          4.5420000e+03, 3.4700000e+02, 7.2440000e+03]],

        [[1.6

# Hardware execution verification

In [1]:
from finn.core.modelwrapper import ModelWrapper
import numpy as np

model = ModelWrapper("/workspace/results/syn_baseline_v3/end2end_quartznet_bitfile.onnx")

add_node = model.get_nodes_by_op_type("Add")
add_input1 = model.get_initializer(add_node[0].input[1])

np.save("/tmp/add_node_val.npy", add_input1)

In [2]:
def execute_final_steps(sdp_out):
    add_node_val = np.load("/tmp/add_node_val.npy")

    sdp_out = np.transpose(sdp_out, [0, 3, 1, 2])
    sdp_out = sdp_out + add_node_val
    sdp_out = np.transpose(sdp_out, [0, 2, 1, 3])
    sdp_out = np.argmax(sdp_out, 2)
    
    return sdp_out

In [5]:
import numpy as np
from finn.core.modelwrapper import ModelWrapper

#sdp_out = np.load("/workspace/results/syn_baseline_v3/output_fpga.npy")
sdp_out = np.load("/workspace/results/syn_optimized_buffer/outputs_test.npy")
print(np.shape(sdp_out))
sdp_out = execute_final_steps(sdp_out)
print("RTL sim: {}".format(__ctc_decoder_predictions_tensor(sdp_out.flatten(), labels)))


out_rtlsim_s1_pt0 = np.load("/workspace/results/syn_baseline_v3/librispeech_data/output_sample_1_pt0.npy")
out_rtlsim_s1_pt1 = np.load("/workspace/results/syn_baseline_v3/librispeech_data/output_sample_1_pt1.npy")
out_rtlsim_s1_pt2 = np.load("/workspace/results/syn_baseline_v3/librispeech_data/output_sample_1_pt2.npy")
out_rtlsim_s1_pt3 = np.load("/workspace/results/syn_baseline_v3/librispeech_data/output_sample_1_pt3.npy")


tests = [out_rtlsim_s1_pt0, out_rtlsim_s1_pt1, out_rtlsim_s1_pt2, out_rtlsim_s1_pt3]
string=[]
for el in tests:
    out_qn = execute_final_steps(el)
    substring = __ctc_decoder_predictions_tensor(out_qn.flatten(),labels)
    string.append(substring)
    
print(string)


(1, 128, 1, 29)
RTL sim: ['vanity and vexation of spat']
[['the spring was abroad in the land'], ["areilla's sober middle age step was"], ['ghter and swifter because of its deep prim'], ['gladness ']]


# Visual tests

In [28]:
from finn.util.visualization import showInNetron
showInNetron("/workspace/results/syn_baseline_v3/end2end_quartznet_bitfile.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/syn_baseline_v3/end2end_quartznet_bitfile.onnx' at http://0.0.0.0:8081


In [3]:
from finn.util.visualization import showInNetron

showInNetron("/workspace/results/syn_baseline/dataflow_partition0_81fmxkyy/df_model_fifos.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/syn_baseline/dataflow_partition0_81fmxkyy/df_model_fifos.onnx' at http://0.0.0.0:8081


In [2]:
showInNetron("/workspace/results/temp/bnn_pynq.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/temp/bnn_pynq.onnx' at http://0.0.0.0:8081


In [2]:
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.fpgadataflow.annotate_cycles import AnnotateCycles
from finn.custom_op.registry import getCustomOp

model = ModelWrapper("/workspace/results/syn_baseline/dataflow_partition0_81fmxkyy/df_model_fifos.onnx")
model = model.transform(AnnotateCycles())

max_cyc = 0
for n in model.graph.node:
    cycles = getCustomOp(n).get_nodeattr("cycles_estimate")
    print("{}\t{}".format(n.name, cycles))
    if cycles>max_cyc:
        max_cyc=cycles
    
print("MAX: {}".format(max_cyc))

StreamingFCLayer_Batch_0	131072
StreamingDataWidthConverter_Batch_0	0
DuplicateStreams_Batch_0	32768
StreamingDataWidthConverter_Batch_1	0
FMPadding_Batch_0	40960
StreamingDataWidthConverter_Batch_2	0
StreamingFCLayer_Batch_1	524288
StreamingDataWidthConverter_Batch_3	0
ConvolutionInputGenerator1D_0	280576
Vector_Vector_Activate_Batch_0	270336
StreamingFCLayer_Batch_2	524288
StreamingDataWidthConverter_Batch_4	0
FMPadding_Batch_1	40960
StreamingDataWidthConverter_Batch_5	0
ConvolutionInputGenerator1D_1	280576
Vector_Vector_Activate_Batch_1	270336
StreamingFCLayer_Batch_3	524288
StreamingDataWidthConverter_Batch_6	0
FMPadding_Batch_2	40960
StreamingDataWidthConverter_Batch_7	0
ConvolutionInputGenerator1D_2	280576
Vector_Vector_Activate_Batch_2	270336
StreamingFCLayer_Batch_4	524288
StreamingDataWidthConverter_Batch_8	0
FMPadding_Batch_3	40960
StreamingDataWidthConverter_Batch_9	0
ConvolutionInputGenerator1D_3	280576
Vector_Vector_Activate_Batch_3	270336
StreamingFCLayer_Batch_5	524288
S

# Verify outputs

In [9]:
labels = [" ", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m",
         "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "'"]
    
def __ctc_decoder_predictions_tensor(tensor, labels):
    """
    Decodes a sequence of labels to words
    """
    blank_id = len(labels)
    hypotheses = []
    labels_map = dict([(i, labels[i]) for i in range(len(labels))])
    decoded_prediction = []
    previous = len(labels)  # id of a blank symbol
    prediction = tensor
    for p in prediction:
        if (p != previous or previous == blank_id) and p != blank_id:
            decoded_prediction.append(p)
        previous = p
    hypothesis = ''.join([labels_map[c] for c in decoded_prediction])
    hypotheses.append(hypothesis)
    return hypotheses

In [5]:
import numpy as np

golden = np.load("/workspace/results/librispeech_data/librispeech_data/output_sample_float_0.npy", allow_pickle=True)
out_export = np.load("/workspace/results/librispeech_data/librispeech_data/quartznet_export_output.npy", allow_pickle=True)
out_tidy = dict(np.load("/workspace/results/librispeech_data/librispeech_data/quartznet_tidy_and_change_shape_tensors.npy", allow_pickle=True).tolist())
out_streamline = dict(np.load("/workspace/results/librispeech_data/librispeech_data/quartznet_streamline.npy", allow_pickle=True).tolist())
out_cpp = dict(np.load("/workspace/results/librispeech_data/librispeech_data/quartznet_cppsim.npy", allow_pickle=True).tolist())

print("Golden: {}".format(__ctc_decoder_predictions_tensor(golden.flatten(), labels)))
print("Exported: {}".format(__ctc_decoder_predictions_tensor(out_export.flatten(), labels)))
print("Tidy: {}".format(__ctc_decoder_predictions_tensor(out_tidy["global_out"].flatten(), labels)))
print("Streamline: {}".format(__ctc_decoder_predictions_tensor(out_streamline["global_out"].flatten(), labels)))
print("CPPSim: {}".format(__ctc_decoder_predictions_tensor(out_cpp["global_out"].flatten(), labels)))

Golden: ['vanity and vexation ev spad']
Exported: ['vanity and vexation of spad']
Tidy: ['vanity and vexation of spad']
Streamline: ['vanity and vexation of spat']
CPPSim: ['vanity and vexation of spat']


In [10]:
import numpy as np

#out_rtlsim = np.load("/workspace/results/syn_baseline_v3/librispeech_data/quartznet_stitched_ip_rtlsim.npy", allow_pickle=True).item()
out_rtlsim = np.load("/workspace/results/syn_optimized_buffer/quartznet_stitched_ip_rtlsim.npy", allow_pickle=True).item()

print("RTLsim: {}".format(__ctc_decoder_predictions_tensor(out_rtlsim["global_out"].flatten(), labels)))

RTLsim: ['vanity and vexation of spat']


## VVAU weights/thresholds


In [16]:
import numpy as np
from finn.core.modelwrapper import ModelWrapper


model = ModelWrapper("/workspace/results/syn_baseline/dataflow_partition0_81fmxkyy/df_model_fifos.onnx")

for n in model.graph.node:
    if n.name=="Vector_Vector_Activate_Batch_32":
        weights = model.get_initializer(n.input[1])
        thresh = model.get_initializer(n.input[2])
        
for n in model.graph.node:
    if n.op_type=="Vector_Vector_Activate_Batch" and n.name!="Vector_Vector_Activate_Batch_32":
        w = model.get_initializer(n.input[1])
        t = model.get_initializer(n.input[2])
        if np.array_equal(weights, w):
            print(n.name)
            print("True")
        if np.array_equal(thresh, t):
            print(n.name)
            print("True")

In [19]:
for i in range(199):
    print('"StreamingDataWidthConverter_Batch_'+str(i)+'" : ' + '{"slr" : -},')

"StreamingDataWidthConverter_Batch_0" : {"slr" : -},
"StreamingDataWidthConverter_Batch_1" : {"slr" : -},
"StreamingDataWidthConverter_Batch_2" : {"slr" : -},
"StreamingDataWidthConverter_Batch_3" : {"slr" : -},
"StreamingDataWidthConverter_Batch_4" : {"slr" : -},
"StreamingDataWidthConverter_Batch_5" : {"slr" : -},
"StreamingDataWidthConverter_Batch_6" : {"slr" : -},
"StreamingDataWidthConverter_Batch_7" : {"slr" : -},
"StreamingDataWidthConverter_Batch_8" : {"slr" : -},
"StreamingDataWidthConverter_Batch_9" : {"slr" : -},
"StreamingDataWidthConverter_Batch_10" : {"slr" : -},
"StreamingDataWidthConverter_Batch_11" : {"slr" : -},
"StreamingDataWidthConverter_Batch_12" : {"slr" : -},
"StreamingDataWidthConverter_Batch_13" : {"slr" : -},
"StreamingDataWidthConverter_Batch_14" : {"slr" : -},
"StreamingDataWidthConverter_Batch_15" : {"slr" : -},
"StreamingDataWidthConverter_Batch_16" : {"slr" : -},
"StreamingDataWidthConverter_Batch_17" : {"slr" : -},
"StreamingDataWidthConverter_Batch_18"

# Apply SLR partitioning & count SDF nodes

In [42]:
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.general import ApplyConfig
from finn.transformation.fpgadataflow.floorplan import Floorplan
from finn.transformation.fpgadataflow.create_dataflow_partition import CreateDataflowPartition
from finn.util.basic import get_by_name

model = ModelWrapper("/workspace/results/syn_baseline/dataflow_partition0_81fmxkyy/df_model_fifos.onnx")
model = model.transform(ApplyConfig("/workspace/results/syn_baseline/folding_config_slr.json"))

for n in model.graph.node:
    slr = get_by_name(n.attribute, "slr").i
    name = n.name
    print("{}\t{}".format(name, slr))
    
model = model.transform(Floorplan())
model = model.transform(CreateDataflowPartition())

print(len(model.graph.node))

model.save("/tmp/slr.onnx")

    

StreamingFCLayer_Batch_0	0
DuplicateStreams_Batch_0	0
StreamingDataWidthConverter_Batch_0	0
FMPadding_Batch_0	0
StreamingDataWidthConverter_Batch_1	0
StreamingFCLayer_Batch_1	0
StreamingDataWidthConverter_Batch_2	0
ConvolutionInputGenerator1D_0	0
Vector_Vector_Activate_Batch_0	0
StreamingFCLayer_Batch_2	0
StreamingDataWidthConverter_Batch_3	0
FMPadding_Batch_1	0
StreamingDataWidthConverter_Batch_4	0
ConvolutionInputGenerator1D_1	0
Vector_Vector_Activate_Batch_1	0
StreamingFCLayer_Batch_3	0
StreamingDataWidthConverter_Batch_5	0
FMPadding_Batch_2	1
StreamingDataWidthConverter_Batch_6	1
ConvolutionInputGenerator1D_2	1
Vector_Vector_Activate_Batch_2	1
StreamingFCLayer_Batch_4	1
StreamingDataWidthConverter_Batch_7	1
FMPadding_Batch_3	1
StreamingDataWidthConverter_Batch_8	1
ConvolutionInputGenerator1D_3	1
Vector_Vector_Activate_Batch_3	1
StreamingFCLayer_Batch_5	1
StreamingDataWidthConverter_Batch_9	1
FMPadding_Batch_4	1
StreamingDataWidthConverter_Batch_10	1
ConvolutionInputGenerator1D_4	1


DuplicateStreams_Batch_0	0
StreamingDataWidthConverter_Batch_0	0
FMPadding_Batch_0	0
StreamingDataWidthConverter_Batch_1	0
StreamingFCLayer_Batch_1	0
StreamingDataWidthConverter_Batch_2	0
ConvolutionInputGenerator1D_0	0
Vector_Vector_Activate_Batch_0	0
StreamingFCLayer_Batch_2	0
StreamingDataWidthConverter_Batch_3	0
FMPadding_Batch_1	0
StreamingDataWidthConverter_Batch_4	0
ConvolutionInputGenerator1D_1	0
Vector_Vector_Activate_Batch_1	0
StreamingFCLayer_Batch_3	0
StreamingDataWidthConverter_Batch_5	0
0-------
StreamingDataWidthConverter_Batch_6	1
ConvolutionInputGenerator1D_2	1
Vector_Vector_Activate_Batch_2	1
StreamingFCLayer_Batch_4	1
StreamingDataWidthConverter_Batch_7	1
FMPadding_Batch_3	1
StreamingDataWidthConverter_Batch_8	1
ConvolutionInputGenerator1D_3	1
Vector_Vector_Activate_Batch_3	1
StreamingFCLayer_Batch_5	1
StreamingDataWidthConverter_Batch_9	1
FMPadding_Batch_4	1
StreamingDataWidthConverter_Batch_10	1
ConvolutionInputGenerator1D_4	1
Vector_Vector_Activate_Batch_4	1
Strea

DuplicateStreams_Batch_8	2
StreamingDataWidthConverter_Batch_97	2
FMPadding_Batch_40	2
StreamingDataWidthConverter_Batch_98	2
StreamingFCLayer_Batch_49	2
StreamingDataWidthConverter_Batch_99	2
ConvolutionInputGenerator1D_40	2
Vector_Vector_Activate_Batch_40	2
StreamingFCLayer_Batch_50	2
StreamingDataWidthConverter_Batch_100	2
FMPadding_Batch_41	2
StreamingDataWidthConverter_Batch_101	2
ConvolutionInputGenerator1D_41	2
Vector_Vector_Activate_Batch_41	2
StreamingFCLayer_Batch_51	2
StreamingDataWidthConverter_Batch_102	2
FMPadding_Batch_42	2
StreamingDataWidthConverter_Batch_103	2
ConvolutionInputGenerator1D_42	2
Vector_Vector_Activate_Batch_42	2
StreamingFCLayer_Batch_52	2
StreamingDataWidthConverter_Batch_104	2
FMPadding_Batch_43	2
StreamingDataWidthConverter_Batch_105	2
ConvolutionInputGenerator1D_43	2
Vector_Vector_Activate_Batch_43	2
StreamingFCLayer_Batch_53	2
StreamingDataWidthConverter_Batch_106	2
FMPadding_Batch_44	2
StreamingDataWidthConverter_Batch_107	2
ConvolutionInputGenerat

StreamingDataWidthConverter_Batch_192	0
FMPadding_Batch_74	0
StreamingDataWidthConverter_Batch_193	0
ConvolutionInputGenerator1D_74	0
Vector_Vector_Activate_Batch_74	0
StreamingDataWidthConverter_Batch_194	0
20-------
StreamingDataWidthConverter_Batch_195	0
AddStreams_Batch_14	0
Thresholding_Batch_14	0
FMPadding_Batch_75	0
StreamingDataWidthConverter_Batch_196	0
ConvolutionInputGenerator1D_75	0
Vector_Vector_Activate_Batch_75	0
StreamingDataWidthConverter_Batch_197	0
21-------
StreamingDataWidthConverter_Batch_198	0
22-------
StreamingFCLayer_Batch_93	0
23-------
24


In [84]:
showInNetron("/tmp/slr.onnx")

Stopping http://0.0.0.0:8081
Serving '/tmp/slr.onnx' at http://0.0.0.0:8081


In [86]:
from finn.transformation.general import GiveUniqueNodeNames

model = ModelWrapper("/tmp/slr.onnx")
model = model.transform(GiveUniqueNodeNames())

model.save("/tmp/slr_named.onnx")
showInNetron("/tmp/slr_named.onnx")

Stopping http://0.0.0.0:8081
Serving '/tmp/slr_named.onnx' at http://0.0.0.0:8081


In [85]:
showInNetron("/tmp/finn_dev_mirza/dataflow_partition6_rgmwvhvs/df_model.onnx")

Stopping http://0.0.0.0:8081
Serving '/tmp/finn_dev_mirza/dataflow_partition6_rgmwvhvs/df_model.onnx' at http://0.0.0.0:8081


In [52]:
from finn.core.modelwrapper import ModelWrapper
from finn.custom_op.registry import getCustomOp
from finn.transformation.infer_data_layouts import InferDataLayouts
from finn.transformation.fpgadataflow.insert_iodma import InsertIODMA
from finn.transformation.fpgadataflow.insert_dwc import InsertDWC
from finn.transformation.general import GiveUniqueNodeNames
from finn.transformation.general import GiveReadableTensorNames
from finn.transformation.fpgadataflow.floorplan import Floorplan
from finn.transformation.fpgadataflow.create_dataflow_partition import CreateDataflowPartition

model = ModelWrapper("/workspace/results/syn_baseline/dataflow_partition0_81fmxkyy/df_model_fifos.onnx")

in_tensor = model.graph.node[0].input[0]
model.set_tensor_layout(in_tensor, ["N", "H", "W", "C"])
model = model.transform(InferDataLayouts())
# prepare at global level, then break up into kernels
prep_transforms = [InsertIODMA(512), InsertDWC()]
for trn in prep_transforms:
    model = model.transform(trn)
    model = model.transform(GiveUniqueNodeNames())
    model = model.transform(GiveReadableTensorNames())

print("Running Floorplan...")
model = model.transform(Floorplan(floorplan="/workspace/results/syn_baseline/folding_config_slr_copy.json"))

print("Running CreateDataflowPartition...")
model = model.transform(CreateDataflowPartition())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
# Save model
model.save("/tmp/df_model_vitisbuild.onnx")

Running Floorplan...


/workspace/finn-base/src/finn/transformation/general.py:331: UserWarning: 
No HW configuration for nodes: IODMA_0, IODMA_1
  "\nNo HW configuration for nodes: " + ", ".join(missing_configurations)
/workspace/finn/src/finn/transformation/fpgadataflow/floorplan.py:124: UserWarning: 2 nodes have no entry in the provided floorplan and no default value was set
  + "and no default value was set"


Running CreateDataflowPartition...
0-------
1-------
DuplicateStreams_Batch_0	0
StreamingDataWidthConverter_Batch_0	0
FMPadding_Batch_0	0
StreamingDataWidthConverter_Batch_1	0
StreamingFCLayer_Batch_1	0
StreamingDataWidthConverter_Batch_2	0
ConvolutionInputGenerator1D_0	0
Vector_Vector_Activate_Batch_0	0
StreamingFCLayer_Batch_2	0
StreamingDataWidthConverter_Batch_3	0
FMPadding_Batch_1	0
StreamingDataWidthConverter_Batch_4	0
ConvolutionInputGenerator1D_1	0
Vector_Vector_Activate_Batch_1	0
StreamingFCLayer_Batch_3	0
StreamingDataWidthConverter_Batch_5	0
2-------
StreamingDataWidthConverter_Batch_6	1
ConvolutionInputGenerator1D_2	1
Vector_Vector_Activate_Batch_2	1
StreamingFCLayer_Batch_4	1
StreamingDataWidthConverter_Batch_7	1
FMPadding_Batch_3	1
StreamingDataWidthConverter_Batch_8	1
ConvolutionInputGenerator1D_3	1
Vector_Vector_Activate_Batch_3	1
StreamingFCLayer_Batch_5	1
StreamingDataWidthConverter_Batch_9	1
FMPadding_Batch_4	1
StreamingDataWidthConverter_Batch_10	1
ConvolutionInputG

DuplicateStreams_Batch_8	2
StreamingDataWidthConverter_Batch_97	2
FMPadding_Batch_40	2
StreamingDataWidthConverter_Batch_98	2
StreamingFCLayer_Batch_49	2
StreamingDataWidthConverter_Batch_99	2
ConvolutionInputGenerator1D_40	2
Vector_Vector_Activate_Batch_40	2
StreamingFCLayer_Batch_50	2
StreamingDataWidthConverter_Batch_100	2
FMPadding_Batch_41	2
StreamingDataWidthConverter_Batch_101	2
ConvolutionInputGenerator1D_41	2
Vector_Vector_Activate_Batch_41	2
StreamingFCLayer_Batch_51	2
StreamingDataWidthConverter_Batch_102	2
FMPadding_Batch_42	2
StreamingDataWidthConverter_Batch_103	2
ConvolutionInputGenerator1D_42	2
Vector_Vector_Activate_Batch_42	2
StreamingFCLayer_Batch_52	2
StreamingDataWidthConverter_Batch_104	2
FMPadding_Batch_43	2
StreamingDataWidthConverter_Batch_105	2
ConvolutionInputGenerator1D_43	2
Vector_Vector_Activate_Batch_43	2
StreamingFCLayer_Batch_53	2
StreamingDataWidthConverter_Batch_106	2
FMPadding_Batch_44	2
StreamingDataWidthConverter_Batch_107	2
ConvolutionInputGenerat

StreamingDataWidthConverter_Batch_192	0
FMPadding_Batch_74	0
StreamingDataWidthConverter_Batch_193	0
ConvolutionInputGenerator1D_74	0
Vector_Vector_Activate_Batch_74	0
StreamingDataWidthConverter_Batch_194	0
22-------
StreamingDataWidthConverter_Batch_195	0
AddStreams_Batch_14	0
Thresholding_Batch_14	0
FMPadding_Batch_75	0
StreamingDataWidthConverter_Batch_196	0
ConvolutionInputGenerator1D_75	0
Vector_Vector_Activate_Batch_75	0
StreamingDataWidthConverter_Batch_197	0
23-------
StreamingDataWidthConverter_Batch_198	0
24-------
StreamingFCLayer_Batch_93	0
25-------
